^fill^README.md^here^

- - -

^fill^theory.md^here^

- - -

## Demonstration

### 1. Library imports

Import libraries needed by the calculation. The external libraries used are:

- [numpy](http://www.numpy.org/)

- [DataModelDict](https://github.com/usnistgov/DataModelDict)

- [atomman](https://github.com/usnistgov/atomman)

- [iprPy](https://github.com/usnistgov/iprPy)

In [1]:
# Standard library imports
from __future__ import division, absolute_import, print_function
import os
import sys
import uuid
import glob
import shutil
import random
import datetime
from copy import deepcopy

# http://www.numpy.org/
import numpy as np 

# https://github.com/usnistgov/DataModelDict 
from DataModelDict import DataModelDict as DM

# https://github.com/usnistgov/atomman 
import atomman as am
import atomman.lammps as lmp
import atomman.unitconvert as uc

# https://github.com/usnistgov/iprPy
import iprPy

### 2. Assign values for the calculation's run parameters

#### 2.0 Set the calculation's working directory

In [2]:
calc_name = 'dislocation_monopole'

# Check current working directory
cwd_name = os.path.basename(os.getcwd())

# Change working directory if needed
if cwd_name != calc_name:
    if not os.path.isdir(calc_name):
        os.mkdir(calc_name)
    os.chdir(calc_name)

#### 2.1 Specify system-specific paths

Simple input parameters:

- __lammps_command__ is the LAMMPS command to use.

- __mpi_command__ MPI command for running LAMMPS in parallel. A value of None will run simulations serially.

- __lib_directory__ defines the relative path to the iprPy library. This makes it easier to define paths to reference records later.

In [3]:
lammps_command = 'lmp_mpi'
mpi_command = 'mpiexec -localonly 4'
lib_directory = '../../../library'

#### 2.2 Specify the potenital and elemental symbols

Simple input parameters:

- __potential_name__ gives the name of the potential_LAMMPS reference record in the iprPy library to use for potential. 

- __symbols__ is a list of the elemental model symbols of potential to associate with the unique atom types of system.

Derived parameters:

- __potential_dir_path__ gives the path for the folder containing the artifacts associated with the potential (i.e. eam.alloy file). If an empty string '' is given, the necessary artifacts are taken as being in the working directory. (Here, it is derived from lib_directory and potential_name.)

- __potential_path__ gives the path to the potential_LAMMPS reference record to use. (Here, it is derived from lib_directory and potential_name.)

- __potential__ is the atomman.lammps.Potential representation of a LAMMPS implemented potential to use. (Here, the class is implemented using potential_path and potential_dir_path.)

In [4]:
potential_name = '1999--Mishin-Y--Ni--LAMMPS--ipr1'
symbols = ['Ni']

# -------------- Derived parameters -------------- #
# Define potential_path and potential_dir_path using lib_directory and potential_name
potential_dir_path = os.path.abspath(os.path.join(lib_directory, 'potential_LAMMPS', potential_name))
potential_path = potential_dir_path + '.json'

# Create potential by loading LAMMPS-potential record
potential = lmp.Potential(potential_path, potential_dir_path)

print('Successfully loaded potential', potential)

Successfully loaded potential 1999--Mishin-Y--Ni--LAMMPS--ipr1


#### 2.3 Specify the prototype unit cell system

Simple input parameters:

- __prototype_name__ gives the name of the crystal_prototype reference record in the iprPy library to use for ucell. 

- __box_parameters__ defines the box parameters to scale ucell to. This is a list of either three or six floats
    - __[a, b, c]__ are the orthogonal lattice parameters in length units.
    - __[a, b, c, alpha, beta, gamma]__ are the triclinic lattice parameters in length units and angles in degrees.

Derived parameters:

- __prototype_path__ gives the path to the crystal_prototype reference record to use. (Here, it is derived from lib_directory and prototype_name).

- __ucell__ is an atomman.System representing a fundamental unit cell of the system. (Here, it is loaded from prototype_path file, then scaled using box_parameters).

In [5]:
prototype_name = 'A1--Cu--fcc'
box_parameters = uc.set_in_units([3.52, 3.52, 3.52], 'angstrom')

# -------------- Derived parameters -------------- #
# Define prototype_path using lib_directory and prototype_name
prototype_path = os.path.abspath(os.path.join(lib_directory, 'crystal_prototype', prototype_name+'.json'))

# Create ucell by loading prototype record
ucell = am.load('system_model', prototype_path)[0]

# Rescale ucell using box_parameters
if len(box_parameters) == 3:
    ucell.box_set(a=box_parameters[0], b=box_parameters[1], c=box_parameters[2], scale=True)
elif len(box_parameters) == 6:
    ucell.box_set(a=box_parameters[0], b=box_parameters[1], c=box_parameters[2], scale=True,
                  alpha=box_parameters[3], beta=box_parameters[4], gamma=box_parameters[5])

print(ucell)

avect =  [ 3.520,  0.000,  0.000]
bvect =  [ 0.000,  3.520,  0.000]
cvect =  [ 0.000,  0.000,  3.520]
origin = [ 0.000,  0.000,  0.000]
natoms = 4
natypes = 1
     id |   atype |  pos[0] |  pos[1] |  pos[2]
      0 |       1 |   0.000 |   0.000 |   0.000
      1 |       1 |   0.000 |   1.760 |   1.760
      2 |       1 |   1.760 |   0.000 |   1.760
      3 |       1 |   1.760 |   1.760 |   0.000


#### 2.4 Specify material elastic constants

Simple input parameters:

- __C_dict__ is a dictionary containing the unique elastic constants for the potential and crystal structure defined above. 

Derived parameters

- __C__ is an atomman.ElasticConstants object built from C_dict.

In [6]:
C_dict = {}
C_dict['C11'] = uc.set_in_units(247.86, 'GPa')
C_dict['C12'] = uc.set_in_units(147.83, 'GPa')
C_dict['C44'] = uc.set_in_units(124.84, 'GPa')

# -------------- Derived parameters -------------- #
# Build ElasticConstants object from C_dict terms
C = am.ElasticConstants(**C_dict)

#### 2.5 Specify the defect 

Simple input parameters:

- __dislocation_name__ gives the name of the dislocation_monopole reference record in the iprPy library to use for potential. 

Derived parameters:

- __dislocation_path__ gives the path to the dislocation_monopole reference record to use. (Here, it is derived from lib_directory and dislocation_name).

- __dislocation_model__ is a DataModelDict of a dislocation_monopole record. (Here, the dislocation_path file is loaded as a DataModelDict.)

- __dislocation_kwargs__ is a dictionary containing parameters for generating the defect. Values are extracted from the dislocation_model record and uniquely define a type of stacking fault. Included keywords are:

    - __burgers__ is the crystallographic Burgers vector for the dislocation
    
    - __crystallographic-axes__ specifies how to orient the system. Subelements define each of the x-, y-, and z-axes.
    
    - __atomshift__ is a 3D vector rigid-body shift to apply to atoms in the system. The atomshift vector is relative to the rotated unit cell's box vectors.

In [7]:
dislocation_name = 'A1--Cu--fcc--111-edge'

# -------------- Derived parameters -------------- #
# Define dislocation_path using lib_directory and dislocation_name
dislocation_path = os.path.abspath(os.path.join(lib_directory, 'dislocation_monopole', dislocation_name+'.json')) 

# Load dislocation-monopole record as a DataModelDict
with open(dislocation_path) as f:
    dislocation_model = DM(f)

print('Successfully loaded defect record for', dislocation_model['dislocation-monopole']['id'])

# Extract defect parameters    
dislocation_kwargs = dislocation_model['dislocation-monopole']['calculation-parameter']

print(dislocation_kwargs.json(indent=4))

Successfully loaded defect record for A1--Cu--fcc--111-edge
{
    "x_axis": " 1  0 -1", 
    "y_axis": " 1  1  1", 
    "z_axis": " 1 -2  1", 
    "atomshift": " 0.00 0.25 0.00", 
    "burgersvector": " 0.5  0.0 -0.5"
}


#### 2.6 Generate the initial system

Simple input parameters:

- __sizemults__ list of three integers or three tuple pairs of integers specifying how many times (and in which direction) the ucell vectors of $a$, $b$ and $c$ are replicated in creating system.

Defect model parameters:

- __x_axis__ is the 3D crystal vector relative to ucell to align with the x-axis of system.

- __y_axis__ is the 3D crystal vector relative to ucell to align with the y-axis of system.

- __z_axis__ is the 3D crystal vector relative to ucell to align with the z-axis of system.

- __atomshift__ is a 3D vector rigid-body shift to apply to atoms in the system. The atomshift vector is relative to the rotated unit cell's box vectors.

Derived parameters:

- __axes__ is a 3x3 array combining x_axis, y_axis and z_axis.

- __shift__ is a 3D vector rigid-body shift to apply to atoms in the system. The shift vector is in absolute Cartesian units. (Here, it is derived from atomshift).

- __system__ is the base atomman.System to use. (Here, it is derived by copying ucell, rotating according to axes, applying shift, then sizemults

In [8]:
sizemults = [(-15,15), (-12,12), (0,2)]

# -------------- Defect parameters --------------- #
x_axis = np.array(dislocation_kwargs['x_axis'].split(), dtype=float)
y_axis = np.array(dislocation_kwargs['y_axis'].split(), dtype=float)
z_axis = np.array(dislocation_kwargs['z_axis'].split(), dtype=float)
atomshift = np.array(dislocation_kwargs['atomshift'].split(), dtype=float)

# -------------- Derived parameters -------------- #
# Copy ucell to initialsystem
system = deepcopy(ucell)

# Build axes from x_axis, y_axis and z_axis
axes = np.array([x_axis, y_axis, z_axis])

# Rotate using axes
system = am.rotate_cubic(system, axes)

# Convert atomshift to shift using the rotated system
shift = (atomshift[0] * system.box.avect 
         + atomshift[1] * system.box.bvect 
         + atomshift[2] * system.box.cvect)

# Apply shift
system.atoms_prop(key='pos', value=system.atoms_prop(key='pos') + shift)

# Apply sizemults
system.supersize(*sizemults)

print('# of atoms in system =', system.natoms)

# of atoms in system = 34560


#### 2.7 Specify calculation-specific run parameters

Simple input parameters:

- __boundarywidth__ sets the minimum width of the fixed-atom boundary region. This is taken in units of the a lattice parameter of the unit cell.

- __boundaryshape__ specifies what shape to make the fixed-atom boundary region.

    - __circle__ will create a cylindrical active region.
    
    - __rect__ will create a rectangular active region.

- __annealtemperature__ is the temperature at which to relax (anneal) the dislocation system. If annealtemperature is 0.0, then only a static relaxation will be performed. Default value is 0.0.

- __randomseed__ allows for the random seed used in generating initial atomic velocities for a dynamic relaxation to be specified. This is an integer between 1 and 900000000. Default value is None, which will randomly pick a number in that range.

- __energytolerance__ is the energy tolerance to use during the minimizations. This is unitless.

- __forcetolerance__ is the force tolerance to use during the minimizations. This is in energy/length units.

- __maxiterations__ is the maximum number of minimization iterations to use.

- __maxevaluations__ is the maximum number of minimization evaluations to use.

- __maxatommotion__ is the largest distance that an atom is allowed to move during a minimization iteration. This is in length units. 

Defect model parameters

- __burgersvector__ defines the crystallographic Burgers vector for the dislocation being added.

Derived parameters

- __T__ is the transformation matrix associated with axes.

- __bwidth__ is the minimum boundary width in length units. This is obtained by scaling boundarywidth by the $a$ unit cell box vector.

- __burgers__ is the Burgers vector in absolute Cartesian units. This is obtained by scaling dislocation_burgersvector by the unit cell box vectors, and then transforming to the defined axes.

In [9]:
boundarywidth = 3
boundaryshape = 'circle'

annealtemperature = 0.0
randomseed = None

energytolerance = 0.0
forcetolerance = uc.set_in_units(1e-6, 'eV/angstrom')
maxiterations = 10000
maxevaluations = 100000
maxatommotion = uc.set_in_units(0.01, 'angstrom')

# -------------- Defect parameters --------------- #
burgersvector = np.array(dislocation_kwargs['burgersvector'].split(), float)

# -------------- Derived parameters -------------- #
# Scale and transform the Burgers vector
T = am.tools.axes_check(axes)
burgers = T.dot(burgersvector[0] * ucell.box.avect + 
                burgersvector[1] * ucell.box.bvect + 
                burgersvector[2] * ucell.box.cvect)
# Scale boundary width
bwidth = ucell.box.a * boundarywidth

### 3. Define calculation function(s) and generate template LAMMPS script(s)

#### 3.1 disl_relax.template

In [10]:
with open('disl_relax.template', 'w') as f:
    f.write("""^fill^disl_relax.template^here^""")

#### 3.2 dislocationmonopole()

In [11]:
^fill^calc_dislocation_monopole.py(dislocationmonopole)^here^

#### 3.3 disl_boundary_fix()

In [12]:
^fill^calc_dislocation_monopole.py(disl_boundary_fix)^here^

#### 3.4 disl_relax()

In [13]:
^fill^calc_dislocation_monopole.py(disl_relax)^here^

#### 3.5 anneal_info() 

In [14]:
^fill^calc_dislocation_monopole.py(anneal_info)^here^

### 4. Run calculation function(s)

In [15]:
results_dict = dislocationmonopole(lammps_command, 
                                   system, 
                                   potential,
                                   symbols,  
                                   burgers,
                                   C,
                                   axes = axes,
                                   mpi_command = mpi_command,
                                   etol = energytolerance,
                                   ftol = forcetolerance,
                                   maxiter = maxiterations,
                                   maxeval = maxevaluations,
                                   dmax = maxatommotion,
                                   annealtemp = annealtemperature, 
                                   randomseed = randomseed, 
                                   bwidth = bwidth, 
                                   bshape = boundaryshape)

In [16]:
results_dict.keys()

['symbols_disl',
 'Stroh_K_tensor',
 'symbols_base',
 'E_total_disl',
 'Stroh_preln',
 'dumpfile_base',
 'dumpfile_disl']

### 5. Report results

#### 5.1 Define units for outputting values

- __length_unit__ is the unit of length to display results in.
- __energy_unit__ is the unit of energy to display cohesive energies in.
- __e_A_unit__ is the energy per area to report the surface energy in.

In [17]:
length_unit = 'angstrom'
energy_unit = 'eV'
pressure_unit = 'GPa'
e_l_unit = energy_unit+'/'+length_unit

#### 5.2 Print Stroh method parameters

In [18]:
print('pre-ln factor (alpha) =', uc.get_in_units(results_dict['Stroh_preln'], e_l_unit), e_l_unit)
print('K_tensor =')
print(uc.get_in_units(results_dict['Stroh_K_tensor'], pressure_unit), pressure_unit)

pre-ln factor (alpha) = 0.344228972217 eV/angstrom
K_tensor =
[[ 123.86918865    0.            0.        ]
 [   0.          127.31323026  -12.76027037]
 [   0.          -12.76027037   81.07544348]] GPa


#### 5.3 List dump files

In [19]:
print('Perfect system is saved as', results_dict['dumpfile_base'])
print('Defect system is saved as',  results_dict['dumpfile_disl'])

Perfect system is saved as base.dump
Defect system is saved as disl.dump
